# Sewershed delineation

In this Jupyter notebook for Python, we delineate a sewershed based on a sewer network and census blocks.

## Data requirements

- Sewer mains
  * Sewer gravity mains and possibly force mains.
  * All flowing into one point (head of the sewershed).
  * The lines don't need to be spatially connected in the data, but the processing will assume that all lines flow into a single point.
- US Census blocks as polygons (areas) with demographics as attributes


<div class="alert alert-block alert-warning">
<b>Note:</b>
GRASS GIS 8.4 (currently in preview) is need for new version of <em>v.dissolve</em> tool and new <em>v.fill.holes</em> tool. Available at <a href="https://grass.osgeo.org/download/">grass.osgeo.org, under download</a>.
</div>

## Software setup 

We will use a couple of standard Python packages and GRASS GIS.

For now, the setup here assumes Linux. Instructions for Windows are available at [GRASS GIS Jupyter notebooks wiki page](https://grasswiki.osgeo.org/wiki/GRASS_GIS_Jupyter_notebooks#Running_a_Jupyter_notebook_locally).

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
from pathlib import Path

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

 extension appropriate for the format, e.g. mains.shp. Census blocks should be in data/census directory and named blocks plus a file extension. Either rename the files or modify the code below if ne extension appropriate for the format, e.g. mains.shp. Census blocks should be in data/census directory and named blocks plus a file extension. Either rename the files or modify the code below if ne## Get the data ready

This notebooks needs gravity mains (or all mains) as vector lines and census blocks as vector polygons with attributes. The gravity mains file should be in directory `data/sewers` and should be named `mains` with file extension appropriate for the format, e.g. `mains.shp`. Census blocks should be in `data/census` directory and named `blocks` plus a file extension. Either rename the files or modify the code below if needed.

The paths can be not only directories but also ZIP files. Similarly, the files can also be layers. The names will be passed to GDAL.

In [ ]:
grass_project = "data/network_examples"

To compute the data, we will use a GRASS project (aka location).

In [ ]:
!grass -e -c $data_directory $grass_project

In [ ]:
session = gj.init(grass_project)

We will use vector lines of gravity mains as our sewershed network. Here, we are using Raleigh as an example. We will also use the US 2020 census blocks for North Carolina. The census blocks are polygons (i.e., areas).

We store the names of GRASS vector maps in Python variables and will use the variables from now on.

In [ ]:
gs.run_command("v.import", input="data/sewers_Raleigh/gravity_lines.shp", output="raleigh")
gs.run_command(
    "v.import",
    input="data/sewers_Greenville/gravity_lines.shp",
    output="greenville",
)

In [ ]:
point_file = "wwtp_points"  # Filename without file extension
wwtp_vector = "wwtp"
gs.run_command(
    "v.import", input="data/sewers_Greenville", layer=point_file, output=wwtp_vector
)

To limit the number of census blocks we need to import, we set the computational region to the extent of sewer mains (extended in all directions by 100 map units - feet or meter).

## Select census blocks

To get only the relevant census blocks, we select only census blocks which overlap with the sewer network.

In [ ]:
gs.run_command("g.region", vector="raleigh", grow="5000")
sewer_map = gj.Map(width=750, height=750, use_region=True)
sewer_map.d_background(color="white")
sewer_map.d_vect(map="raleigh", legend_label="Gravity mains", color="#0067CC")
sewer_map.d_vect(
    map=wwtp_vector,
    legend_label="WWTP",
    color="#865A0C",
    fill_color="#FCA636",
    size=10,
    icon="basic/circle",
    where="OWNER = 'City of Raleigh'",
    attribute_column="FACILITY",
    xref="right",
    yref="top",
    label_color="black",
    label_size=12,
    label_bgcolor="white",
    flags="s",
)
sewer_map.d_vect(
    map=wwtp_vector,
    legend_label="WWTP",
    color="#865A0C",
    fill_color="#FCA636",
    size=10,
    icon="basic/circle",
    where="OWNER = 'City of Raleigh'",
)
sewer_map.d_text(text="(a)", color="#111111", size="6", at=(0, 95))
sewer_map.d_legend_vect(flags="b", at=(75, 19), title="Raleigh")
sewer_map.d_barscale(flags="n", at=(65, 7))
sewer_map.save(Path(".") / "raleigh_mains.png")
sewer_map.show()

In [ ]:
gs.run_command("g.region", vector="greenville", grow="5000")
sewer_map = gj.Map(width=750, height=750, use_region=True)
sewer_map.d_background(color="white")
sewer_map.d_vect(map="greenville", legend_label="Gravity mains", color="#0067CC")
sewer_map.d_vect(
    map=wwtp_vector,
    legend_label="WWTP",
    color="#865A0C",
    fill_color="#FCA636",
    size=10,
    icon="basic/circle",
    where="OWNER = 'Greenville Utilities Commission'",
    attribute_column="FACILITY",
    xref="left",
    yref="top",
    label_color="black",
    label_size=12,
)
sewer_map.d_text(text="(b)", color="#111111", size="5", at=(0, 89))
sewer_map.d_legend_vect(flags="b", at=(75, 19), title="Greenville")
sewer_map.d_barscale(flags="n", at=(58, 7))
sewer_map.save(Path(".") / "greenville_mains.png")
sewer_map.show()

In [ ]:
!convert raleigh_mains.png greenville_mains.png +append mains_raleigh_greenville.png
from IPython.display import Image

Image("mains_raleigh_greenville.png")